# Classification d'images de vêtements

Ce Notebook forme un modèle de réseau de neurone pour classer les images de vêtements, comme les baskets et les chemises.

In [ ]:
# TensorFlow et tf.keras
import tensorflow as tf
from tensorflow import keras

# Bibliothèques 
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Importation du jeu de données Fashion MNIST

Ce guide utilise le jeu de données [Fashion MNIST](http://yann.lecun.com/exdb/mnist) qui contient 70 000 images en niveaux de gris dans 10 catégories. Les images montrent des vêtements individuels à basse résolution (28 par 28 pixels), comme on le voit ici :

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="(http://yann.lecun.com/exdb/mnist/)">Fashion-MNIST samples</a>.<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST est conçu pour remplacer l'ensemble de données classique [MNIST](http://yann.lecun.com/exdb/mnist/), souvent utilisé comme "Hello, World" des programmes d'apprentissage automatique pour la vision par ordinateur. . Le jeu de données MNIST contient des images de chiffres manuscrits (0, 1, 2, etc.) dans un format identique à celui des vêtements que vous utiliserez ici.

Ce guide utilise Fashion MNIST pour la variété et parce que c'est un problème légèrement plus difficile que le MNIST ordinaire. Les deux ensembles de données sont relativement petits et sont utilisés pour vérifier qu'un algorithme fonctionne comme prévu. Ce sont de bons points de départ pour tester et déboguer le code.

Ici, 60 000 images sont utilisées pour former le réseau et 10 000 images pour évaluer la précision avec laquelle le réseau a appris à classer les images. Vous pouvez accéder au Fashion MNIST directement depuis TensorFlow. Importez et chargez les données Fashion MNIST directement depuis TensorFlow :

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Le chargement de l'ensemble de données renvoie quatre tableaux NumPy :

* Les tableaux `train_images` et `train_labels` sont l'*ensemble d'apprentissage*, c'est-à-dire les données que le modèle utilise pour apprendre.
* Le modèle est testé par rapport au *test set*, aux tableaux `test_images` et `test_labels`.

Les images sont des tableaux NumPy 28x28, avec des valeurs de pixels allant de 0 à 255. Les *étiquettes* sont un tableau d'entiers, allant de 0 à 9. Ceux-ci correspondent à la *classe* de vêtements que l'image représente :

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Chaque image est associée à une seule étiquette. Étant donné que les *class names* ne sont pas inclus dans l'ensemble de données, stockez-les ici pour les utiliser ultérieurement lors du traçage des images :

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Exploration des données

Explorons le format de l'ensemble de données avant de former le modèle. L'exemple suivant montre qu'il y a 60 000 images dans l'ensemble d'apprentissage, chaque image étant représentée en 28 x 28 pixels :

In [ ]:
train_images.shape

De même, il y a 60 000 libellés dans le training set:

In [ ]:
len(train_labels)

Chaque étiquette est un entier compris entre 0 et 9 :

In [ ]:
train_labels

Il y a 10 000 images dans le jeu de test. Encore une fois, chaque image est représentée en 28 x 28 pixels :

In [ ]:
test_images.shape

Et le jeu de test contient 10 000 étiquettes d'images :

In [ ]:
len(test_labels)

## Prétraiter les données

Les données doivent être prétraitées avant de former le réseau. Si vous inspectez la première image de l'ensemble d'apprentissage, vous verrez que les valeurs de pixel se situent dans la plage de 0 à 255 :

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Mettez ces valeurs à l'échelle dans une plage de 0 à 1 avant de les alimenter au modèle de réseau neuronal. Pour ce faire, divisez les valeurs par 255. Il est important que le *training set* et le *testing set* soient prétraités de la même manière :

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Pour vérifier que les données sont au format correct et que vous êtes prêt à créer et à entraîner le réseau, affichons les 25 premières images du *training set* et affichons le nom de la classe sous chaque image.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Construisons le modèle

Construire le réseau de neurones nécessite de configurer les couches du modèle, puis de compiler le modèle.

### Configurer les calques (layers)

Le bloc de construction de base d'un réseau de neurones est la *couche*. Les couches extraient des représentations des données qui y sont introduites. Espérons que ces représentations sont significatives pour le problème à résoudre.

L'essentiel de l'apprentissage en profondeur consiste à enchaîner des couches simples. La plupart des couches, telles que `tf.keras.layers.Dense`, ont des paramètres qui sont appris lors de la formation.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

La première couche de ce réseau, `tf.keras.layers.Flatten`, transforme le format des images d'un tableau bidimensionnel (de 28 par 28 pixels) en un tableau unidimensionnel (de 28 * 28 = 784 pixels ). Considérez ce calque comme désempilant des rangées de pixels dans l'image et en les alignant. Cette couche n'a aucun paramètre à apprendre ; il ne fait que reformater les données.

Une fois les pixels aplatis, le réseau consiste en une séquence de deux couches `tf.keras.layers.Dense`. Ce sont des couches neuronales densément connectées ou entièrement connectées. La première couche "Dense" comporte 128 nœuds (ou neurones). La deuxième (et dernière) couche renvoie un tableau logits d'une longueur de 10. Chaque nœud contient un score indiquant que l'image actuelle appartient à l'une des 10 classes.

### Compilons le modèle

Avant que le modèle ne soit prêt pour l'entraînement, il a besoin de quelques réglages supplémentaires. Ceux-ci sont ajoutés lors de l'étape de *compilation* du modèle :

* *Fonction de perte* — Ceci mesure la précision du modèle pendant l'entraînement. Vous souhaitez minimiser cette fonction pour "orienter" le modèle dans la bonne direction.
* *Optimiseur* — C'est ainsi que le modèle est mis à jour en fonction des données qu'il voit et de sa fonction de perte.
* *Métriques* — Utilisées pour surveiller les étapes de formation et de test. L'exemple suivant utilise *précision*, la fraction des images correctement classées.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Former le modèle

L'entraînement du modèle de réseau neuronal nécessite les étapes suivantes :

1. Transférez les données d'apprentissage au modèle. Dans cet exemple, les données d'apprentissage se trouvent dans les tableaux `train_images` et `train_labels`.
2. Le modèle apprend à associer des images et des étiquettes.
3. Vous demandez au modèle de faire des prédictions sur un jeu de test — dans cet exemple, le tableau `test_images`.
4. Vérifiez que les prédictions correspondent aux étiquettes du tableau `test_labels`.

### Nourrir le modèle

Pour commencer l'entraînement, appelez la méthode "model.fit", ainsi appelée car elle "adapte" le modèle aux données d'entraînement :

In [ ]:
model.fit(train_images, train_labels, epochs=10)

Au fur et à mesure que le modèle s'entraîne, les métriques de perte et de précision sont affichées. Ce modèle atteint une précision d'environ 0,91 (ou 91%) sur les données d'apprentissage.

### Évalons la précision

Ensuite, comparons les performances du modèle sur l'ensemble de données de test :

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Il s'avère que la précision sur l'ensemble de données de test est un peu inférieure à la précision sur l'ensemble de données d'apprentissage. Cet écart entre la précision de l'entraînement et la précision des tests représente le *surapprentissage*. Le surajustement se produit lorsqu'un modèle d'apprentissage automatique fonctionne moins bien sur de nouvelles entrées inédites que sur les données d'apprentissage. Un modèle surajusté "mémorise" le bruit et les détails dans l'ensemble de données d'apprentissage à un point tel qu'il a un impact négatif sur les performances du modèle sur les nouvelles données.

### Prédictions

Avec le modèle formé, on peut l'utiliser pour faire des prédictions sur certaines images.
Les sorties linéaires du modèle. Attachons une couche softmax pour convertir les logits en probabilités, qui sont plus faciles à interpréter.

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

Ici, le modèle a prédit l'étiquette pour chaque image de l'ensemble de test. Reprenons la première prédiction :

In [ ]:
predictions[0]

Une prédiction est un tableau de 10 nombres. Ils représentent la "confiance" du modèle que l'image correspond à chacun des 10 vêtements différents. Vous pouvez voir quelle étiquette a la valeur de confiance la plus élevée :

In [ ]:
np.argmax(predictions[0])

Ainsi, le modèle est plus sûr que cette image est une bottine, ou `class_names[9]`. L'examen de l'étiquette du test montre que cette classification est correcte :

In [ ]:
test_labels[0]

Représentez-le graphiquement pour examiner l'ensemble complet des 10 prédictions de classe.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

### Vérifions les prédictions

Avec le modèle formé, vous pouvez l'utiliser pour faire des prédictions sur certaines images.

Regardons la 0ème image, les prédictions et le tableau de prédiction. Les étiquettes de prédiction correctes sont bleues et les étiquettes de prédiction incorrectes sont rouges. Le nombre donne le pourcentage (sur 100) pour l'étiquette prédite.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Traçons plusieurs images avec leurs prédictions. Notez que le modèle peut se tromper même lorsqu'il est très confiant.

In [ ]:
# Tracez les premières images de test X, leurs étiquettes prédites et les vraies étiquettes.
# Coloriez les prédictions correctes en bleu et les prédictions incorrectes en rouge.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Utiliser le modèle entraîné

Enfin, utilisez le modèle formé pour faire une prédiction sur une seule image.

In [ ]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

Les modèles `tf.keras` sont optimisés pour faire des prédictions sur un *batch*, ou une collection, d'exemples à la fois. Par conséquent, même si vous utilisez une seule image, vous devez l'ajouter à une liste :

In [ ]:
# Ajoutez l'image à un lot dont elle est le seul membre.
img = (np.expand_dims(img,0))

print(img.shape)

Prédisons maintenant le bon libellé pour cette image :

In [ ]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`keras.Model.predict` renvoie une liste de listes—une liste pour chaque image dans le lot de données. Saisissez les prédictions pour notre (seule) image dans le lot :

In [ ]:
np.argmax(predictions_single[0])

Et le modèle prédit une étiquette comme prévu.